## My first Convolutional neural network

It took some time and most off all some awesome tutorials from pythonprogramming.net but I finally got my first CNN to work.

I use the Celeb image dataset from Kaggle and decided to see if it is possible to train a neural network that will tell you if you are attractive or not. Do you dare to try it?

In [1]:
import numpy as np
import os
import cv2


DATADIR = "C:\\Python projects\\Attractive\\Data"

CATEGORIES = ["NotAttractive","Attractive"]

for category in CATEGORIES:  
    path = os.path.join(DATADIR,category)  
    for img in os.listdir(path):  
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE) 


        break  
    break  

IMG_SIZE = 50

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

training_data = []

def create_training_data():
    for category in CATEGORIES:  

        path = os.path.join(DATADIR,category)  
        class_num = CATEGORIES.index(category)  

        for img in os.listdir(path):  
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                training_data.append([new_array, class_num])  
            except Exception as e:  
                pass


create_training_data()

import random

random.shuffle(training_data)

X = []
y = []

for features,label in training_data:
    X.append(features)
    y.append(label)



X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from sklearn.model_selection import train_test_split

import pickle

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)
X = np.array(X)
pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)


model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=8)

Epoch 1/8
125/125 [==============================] - 18s 141ms/step - loss: 0.6468 - accuracy: 0.6345
Epoch 2/8
125/125 [==============================] - 18s 140ms/step - loss: 0.5449 - accuracy: 0.7295
Epoch 3/8
125/125 [==============================] - 18s 142ms/step - loss: 0.5315 - accuracy: 0.7380
Epoch 4/8
125/125 [==============================] - 18s 143ms/step - loss: 0.4939 - accuracy: 0.7657
Epoch 5/8
125/125 [==============================] - 18s 142ms/step - loss: 0.4792 - accuracy: 0.7707
Epoch 6/8
125/125 [==============================] - 18s 144ms/step - loss: 0.4628 - accuracy: 0.7825
Epoch 7/8
125/125 [==============================] - 17s 140ms/step - loss: 0.4455 - accuracy: 0.7885
Epoch 8/8
125/125 [==============================] - 18s 144ms/step - loss: 0.4242 - accuracy: 0.8018


The accuracy seems to stabelize after 6 epochs but takes a small jump in the last one

In [3]:
model.save("Attractive")

INFO:tensorflow:Assets written to: Attractive\assets


In [4]:
from keras.models import load_model
model = load_model("Attractive")
model.evaluate(x=X_test, y=y_test)

32/32 [==============================] - 1s 36ms/step - loss: 0.5603 - accuracy: 0.7323


[0.5602554082870483, 0.7322677373886108]

As expected the accuracy in the evaluation is slightly lower.